In [2]:
#kaggle 에서 konlpy 설치 
!pip install konlpy

     |████████████████████████████████| 19.4 MB 18.5 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 3.8 MB/s  eta 0:00:01
     |████████████████████████████████| 453 kB 22.1 MB/s eta 0:00:01
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.9.0
    Uninstalling beautifulsoup4-4.9.0:
      Successfully uninstalled beautifulsoup4-4.9.0
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
#필요 패키지 import 

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import sklearn
from sklearn.decomposition import TruncatedSVD
import seaborn as sns
from pandas_profiling import ProfileReport 

import pandas as pd
import re
import pickle
from gensim.models import word2vec
from gensim.models import FastText
from konlpy.tag import Mecab, Kkma, Okt, Komoran



In [4]:
#상품부분정보만 가져옴 
train_df = pd.read_csv('../input/store-datas/data_2.csv', sep='\t', encoding="UTF-16")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
#분석에 사용할 리뷰 텍스트 확인 
train_df['review_text']

0        싼맛에 샀습니다 그냥 뭐 쿠션감도 별로고 그냥 그렇습니다 지방이라 배송비도 7만원 이네요
1        디자인:그냥 사진 그대로임\n\n사이즈 :170 인 나와 150인 아들 앉기엔 널널...
2        신혼집에 들인 듀커소파!\n거실이 작아서 4인용을 하기엔 너무 크고,, 2인용을 하...
3        반려동물 발톱에 뜯기지 않는다고 해서 구매했는데 우리집 냥이가 뜯뜯하면 죄다 뜯길것...
4        <듀커소파 DK052>\n가로길이 176cm, 라이트그레이\n•배송기간 : 하루만에...
                               ...                        
98479                             튼튼하고 예뻐요\n침대옆에 두기 굿굿베리굿~
98480                          사이드테이블로 잘 사용중입니다 조립하는것도 쉬워요
98481    마감처리가 별로에요. 다리부분에 찍힌 자국도 많고 유리가 안들어가길래 봤더니 나무가...
98482    다 설치하고 유리 낄려고 보니깐 저렇게 다 깨져있네요 전화도 안받으시고 어떻게 좀 ...
98483                                 조립 매우쉽고 이뻐요 강추입니다~~~
Name: review_text, Length: 98484, dtype: object

In [8]:
#용량 문제로 일부 데이터만 추출 
train_df.sample(frac=0.4) 

,Unnamed: 0,Unnamed: 0.1,id,name,review_count,review_average,scrap_count,view_count,review_user_id,review_status,...,review_star_durability,review_star_design,review_star_cost,review_star_delivery,review_star_avg,product_info_id,production_info_name,production_info_brand_name,production_info_explain,production_info_is_purchased
53400,53400,53400.0,205192,빈백 904 소파 8colors,181,4.44,6696,81140,6313126.0,확인완료,...,5.0,5.0,2.0,3.0,3.75,205192.0,빈백 904 소파 8colors,폴리몰리,라이트베이지,True
39303,39303,39303.0,177206,아쿠아텍스 패브릭 소파 (3인/4인) 2colors,80,4.32,2613,38050,7913951.0,확인완료,...,5.0,5.0,4.0,5.0,4.75,177206.0,아쿠아텍스 패브릭 소파 (3인/4인) 2colors,노르웨이숲,아쿠아 3인소파 (스카이블루),True
52377,52377,52377.0,111084,빈백 607C 그랜드 소파 12colors,1569,4.53,41438,546110,1237304.0,확인완료,...,5.0,5.0,5.0,5.0,5.00,111084.0,빈백 607C 그랜드 소파 12colors,폴리몰리,모델 색상: 607C 라떼베이지 / 빈백 구성: 빈백 풋스툴세트,True
36902,36902,36902.0,151076,아리아 패브릭 카우치소파 3colors,55,4.66,1498,21369,6518009.0,확인완료,...,5.0,5.0,5.0,5.0,5.00,151076.0,아리아 패브릭 카우치소파 3colors,조은가구,네이비블루 B형,True
28945,28945,28945.0,126537,쟈드 3인용 가죽소파 2colors,95,4.57,2137,36299,2120359.0,확인완료,...,4.0,4.0,3.0,3.0,3.50,126537.0,쟈드 3인용 가죽소파 2colors,자네트가구,구성: 3인용소파+스툴 / 색상: 블랙,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25306,25306,25306.0,86597,[쿠폰할인] 소이트 3인+카우치 스타일클린 워셔블 패브릭소파 2type(추가커버할인),132,4.64,7239,119615,2692433.0,확인완료,...,0.0,0.0,0.0,0.0,5.00,86597.0,[쿠폰할인] 소이트 3인+카우치 스타일클린 워셔블 패브릭소파 2type(추가커버할인),버즈가구,소이트 3인+카우치 워셔블 패브릭소파 2종 택1,False
25446,25446,25446.0,267172,[쿠폰할인] DK010 4인용 풀커버 그레이 패브릭 발수소파 (1+1 커버증정),88,4.72,2290,30292,2866859.0,확인완료,...,0.0,0.0,0.0,0.0,5.00,267172.0,[쿠폰할인] DK010 4인용 풀커버 그레이 패브릭 발수소파 (1+1 커버증정),듀커소파,NaN,False
40625,40625,40625.0,107831,신소재 FTL 패브릭레더 SMEG 소파 2colors (3인),39,4.24,1706,32298,1800176.0,확인완료,...,4.0,4.0,4.0,1.0,3.25,107831.0,신소재 FTL 패브릭레더 SMEG 소파 2colors (3인),mont(몬트),3인용소파 아이보리,True
48827,48827,48827.0,270528,아반티 와이드 3인 기능성 카시미라 패브릭 소파,3,5.00,18,335,2692433.0,확인완료,...,0.0,0.0,0.0,0.0,5.00,270528.0,아반티 와이드 3인 기능성 카시미라 패브릭 소파,ESSA,아반티 와이드 3인 기능성 카시미라 패브릭 소파,False


In [9]:
#전체 데이터 정보 
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98484 entries, 0 to 98483
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    98484 non-null  int64  
 1   Unnamed: 0.1                  59898 non-null  float64
 2   id                            98484 non-null  int64  
 3   name                          98484 non-null  object 
 4   review_count                  98484 non-null  int64  
 5   review_average                98484 non-null  float64
 6   scrap_count                   98484 non-null  int64  
 7   view_count                    98484 non-null  int64  
 8   review_user_id                96290 non-null  float64
 9   review_status                 96290 non-null  object 
 10  review_text                   96288 non-null  object 
 11  review_star_durability        96290 non-null  float64
 12  review_star_design            96290 non-null  float64
 13  r

In [10]:
#na정보 삭제 
train_df2 = train_df.dropna(axis=0, subset = ['review_user_id', 'review_status', 'review_text', 'review_star_durability', 'review_star_design', 
                                   'review_star_cost', 'review_star_delivery', 'review_star_avg', 'product_info_id', 
                                   'production_info_name', 'production_info_brand_name', 'production_info_explain','production_info_is_purchased'])

In [11]:
#na 정보가 없는지 확인 
train_df2.isnull().sum()

Unnamed: 0                          0
Unnamed: 0.1                    36570
id                                  0
name                                0
review_count                        0
review_average                      0
scrap_count                         0
view_count                          0
review_user_id                      0
review_status                       0
review_text                         0
review_star_durability              0
review_star_design                  0
review_star_cost                    0
review_star_delivery                0
review_star_avg                     0
product_info_id                     0
production_info_name                0
production_info_brand_name          0
production_info_explain             0
production_info_is_purchased        0
dtype: int64

In [12]:
#불필요 정보 삭제 
train_df2 = train_df2.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [13]:
# 형태소 분석기 선언

kkma = Kkma()


In [14]:
# 전처리 함수 선언
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

def remove_brackets(text):
    return re.sub("[\(\[].*?[\)\]]", "", text)

def remove_punc(text):
    return re.sub("[!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~]", ' ', text)

def keep_korean(text):
    return re.sub(r"[^가-힣]", ' ', text)

def pos(text):
    return [token for token, tag in kkma.pos(text) if tag[:2] == 'NN' or tag[:2] == 'NP' or tag[:2] == 'SL']


In [15]:
#분석할 대상인 review 텍스트만 저장 
reviews = train_df2['review_text']

In [18]:
reviews[:20]

0     싼맛에 샀습니다 그냥 뭐 쿠션감도 별로고 그냥 그렇습니다 지방이라 배송비도 7만원 이네요
1     디자인:그냥 사진 그대로임\n\n사이즈 :170 인 나와 150인 아들 앉기엔 널널...
2     신혼집에 들인 듀커소파!\n거실이 작아서 4인용을 하기엔 너무 크고,, 2인용을 하...
3     반려동물 발톱에 뜯기지 않는다고 해서 구매했는데 우리집 냥이가 뜯뜯하면 죄다 뜯길것...
4     <듀커소파 DK052>\n가로길이 176cm, 라이트그레이\n•배송기간 : 하루만에...
6     CS가 엉망입니다.\n\n쇼파 자체는 매우 만족합니다. 하지만, 배송기사의 태도에 ...
7     보자마자 너무예뻐서 바로 구입한 쇼파!\n베이지 색깔과 우드 다리가 넘 예뻐보였는데...
8     일요일 밤 11시 50분에 주문했는데 다음날 저녁에 바로 도착했어요ㅋㅋㅋ가구 배송이...
9                  너무이뻐요 미쳤어요!! 너무 이쁩니댜앙 매일 보며 힐링중입니당 ㅎ
10                      두번 세번 구매허세요ㅎ 튼튼하고 예쁘고 실용성도 굿입니다
11    원하던 사이즈, 원하던 디자인입니다ㅎㅎㅎ\n베이지로 하려다가 라이트 그레이로 바꿨는...
12    아담한 소파 찾고 있었는데 20평대에 딱 맞는 것 같아요. 색상도 고민 많이 했었는...
13    집이작아서 적당한길이의 소파를찾고있었습니다.\n그리고 고양이를 키워서 세탁이가능하거...
14    저희집이 엘베없는4층이라서 기사님 넘힘드셨을텐데\n너무친절하셨어요!\n배송도빠르고 ...
16    20평대아파트에 거실이 좁아서 \n접이식쇼파로 생활했었는데요 \n숨이다죽고 앉았다일...
17    설치했더니 아들이 좋다고 누워서 안 일어나네요~\n생각했던것처럼 튼튼하고 패브릭이라...
18    정말 정말 맘에 드는 소파라 모두 구입하시라고 추천하고 싶어요~ 금요일에 주문했는데...
19    전체 커버를 벗길 수 있다는게 최고 장점이에요\n설치해주시는 기사님이 커버 

In [20]:
#object형태를 str로 변형하여 저장 
review_list = [str(sentence) for sentence in reviews]

In [32]:
#데이터 분석을 위해 하나의 리뷰를 리스트 단위로 묶음 
review_token_list2 = [sentence.split('\',') for sentence in review_list]

In [34]:
#토큰화하여 저장
review_token_list3 = list()
for line in review_token_list2:
    sen =list()
    for s in line:
        sen.append(s.split())
    review_token_list3.append(sen)

In [37]:
#중첩 배열 처리된 것을 2차 배열로 재처리
review_token_list4=list()
for w in review_token_list3:
    for a in w:
        review_token_list4.append(a)

In [38]:
review_token_list4[:3]

[['싼맛에',
  '샀습니다',
  '그냥',
  '뭐',
  '쿠션감도',
  '별로고',
  '그냥',
  '그렇습니다',
  '지방이라',
  '배송비도',
  '7만원',
  '이네요'],
 ['디자인:그냥',
  '사진',
  '그대로임',
  '사이즈',
  ':170',
  '인',
  '나와',
  '150인',
  '아들',
  '앉기엔',
  '널널',
  '1명',
  '더',
  '앉음',
  '좀',
  '작음',
  '208cm임에도',
  '불구하고',
  '착석:리뷰같이',
  '단단함',
  '첨',
  '앉고는',
  '헉',
  '했지만',
  '오래',
  '앉으면',
  '꺼지는',
  '것보단',
  '탄탄하고',
  '쿠션을',
  '치워야',
  '괜찮지',
  '쿠션끼고',
  '앉기엔',
  '폭이',
  '좀',
  '작음',
  '뒷받침이',
  '긴',
  '쇼파가',
  '아니므로',
  '쿠션을',
  '올려야',
  '만족스런',
  '기댐이',
  '가능',
  '색상:베이지라',
  '광목천',
  '대고',
  '쓸예정이고',
  '밝은',
  '색이라',
  '19평같은',
  '집인',
  '우리집엔',
  '좋음',
  '그레이색은',
  '첨부터',
  '제외한지라',
  '만족',
  '주의사항같이',
  '붉은',
  '끼가',
  '몇군데',
  '보여',
  '싼맛에',
  '그냥두려',
  '했지만',
  '물티슈로',
  '쓱싹딱으니',
  '잘',
  '지워짐',
  '착용감:',
  '이',
  '지긋지긋한',
  '장마엔',
  '좀',
  '덥지만',
  '쫙',
  '달라붙는',
  '가죽이나',
  '레자보단',
  '좋으니',
  '이것또한',
  '만족',
  '가격:배송비가',
  '배보다',
  '배꼽이',
  '크지만',
  '(나는',
  '치즈가',
  '유명한',
  '지역에',
  '사는',
  '사람)7만원',
  '내고',
  '사야하는',
  '지

In [27]:
review_token_list4[:4]

[['싼맛에',
  '샀습니다',
  '그냥',
  '뭐',
  '쿠션감도',
  '별로고',
  '그냥',
  '그렇습니다',
  '지방이라',
  '배송비도',
  '7만원',
  '이네요'],
 ['디자인:그냥',
  '사진',
  '그대로임',
  '사이즈',
  ':170',
  '인',
  '나와',
  '150인',
  '아들',
  '앉기엔',
  '널널',
  '1명',
  '더',
  '앉음',
  '좀',
  '작음',
  '208cm임에도',
  '불구하고',
  '착석:리뷰같이',
  '단단함',
  '첨',
  '앉고는',
  '헉',
  '했지만',
  '오래',
  '앉으면',
  '꺼지는',
  '것보단',
  '탄탄하고',
  '쿠션을',
  '치워야',
  '괜찮지',
  '쿠션끼고',
  '앉기엔',
  '폭이',
  '좀',
  '작음',
  '뒷받침이',
  '긴',
  '쇼파가',
  '아니므로',
  '쿠션을',
  '올려야',
  '만족스런',
  '기댐이',
  '가능',
  '색상:베이지라',
  '광목천',
  '대고',
  '쓸예정이고',
  '밝은',
  '색이라',
  '19평같은',
  '집인',
  '우리집엔',
  '좋음',
  '그레이색은',
  '첨부터',
  '제외한지라',
  '만족',
  '주의사항같이',
  '붉은',
  '끼가',
  '몇군데',
  '보여',
  '싼맛에',
  '그냥두려',
  '했지만',
  '물티슈로',
  '쓱싹딱으니',
  '잘',
  '지워짐',
  '착용감:',
  '이',
  '지긋지긋한',
  '장마엔',
  '좀',
  '덥지만',
  '쫙',
  '달라붙는',
  '가죽이나',
  '레자보단',
  '좋으니',
  '이것또한',
  '만족',
  '가격:배송비가',
  '배보다',
  '배꼽이',
  '크지만',
  '(나는',
  '치즈가',
  '유명한',
  '지역에',
  '사는',
  '사람)7만원',
  '내고',
  '사야하는',
  '지

In [ ]:
#데이터 전처리
review = list()
for rev in review_token_list4:
    s = list()
    for sen in rev:
        s.append(remove_tags(sen))
    review.append(s)


In [ ]:
#데이터 전처리
review2 = list()
for rev in review:
    s = list()
    for sen in rev:
        s.append(remove_brackets(sen))
    review2.append(s)


In [ ]:
#데이터 전처리
review3 = list()
for rev in review2:
    s = list()
    for sen in rev:
        s.append(remove_punc(sen))
    review3.append(s)


In [ ]:
#데이터 전처리
review4 = list()
for rev in review3:
    s = list()
    for sen in rev:
        s.append(keep_korean(sen))
    review4.append(s)


In [ ]:
#용량 문제로 일부만 여러 개 추출하여 사용(한 번에 돌릴 수 있는 정도로
#random으로 추출시 오류 발생하여 임의 구간 추출
review_t1 = review4[:20000:55]
review_t2 = review4[20000:40000:55]
review_t3 = review4[40000:60000:55]
review_t4 = review4[60000:80000:55]

In [ ]:
#kkma pos 태깅시 추출 데이터 확인
reviewPos_test = list()
for rev in review_t1[:10]:
    test=[]
    all =""
    for sen in rev:
        all += sen
    for s,tag in kkma.pos(all):
        if tag[:5] == 'NN' or tag[:5] == 'VA' or tag[:5] == 'VV':
            test = [s,tag]
            reviewPos_test.append(test)
    

In [ ]:
#komoran 추출 데이터 확인. 속도 문제로 나눠서 처리
qq = list()
for rev in review_t1:
    all =""
    for sen in rev:
        all += sen
    qq.append(Komoran.nouns(all))
    

In [ ]:
qq = list()
for rev in review_t2:
    all =""
    for sen in rev:
        all += sen
    qq.append(Komoran.nouns(all))
    

In [ ]:

qq = list()
for rev in review_t3:
    all =""
    for sen in rev:
        all += sen
    qq.append(Komoran.nouns(all))
    

In [ ]:

qq = list()
for rev in review_t4:
    all =""
    for sen in rev:
        all += sen
    qq.append(Komoran.nouns(all))
    

In [ ]:
#kkma morphs추출 데이터 확인
reviewPos_test2 = list()
for rev in review_t1[:50]:
    all =""
    for sen in rev:
        all += sen
    reviewPos_test2.append(kkma.morphs(all))  

In [ ]:
#데이터 처리 속도 문제로 나눠서 처리
reviewPos = list()
for rev in review_t1:
    all =""
    for sen in rev:
        all += sen
    reviewPos.append(kkma.nouns(all))
    

In [ ]:
#데이터 처리 속도 문제로 나눠서 처리
for rev in review_t2:
    all =""
    for sen in rev:
        all += sen
    reviewPos.append(kkma.nouns(all))

In [ ]:
#데이터 처리 속도 문제로 나눠서 처리
for rev in review_t3:
    all =""
    for sen in rev:
        all += sen
    reviewPos.append(kkma.nouns(all))

In [ ]:
#데이터 처리 속도 문제로 나눠서 처리
for rev in review_t4:
    all =""
    for sen in rev:
        all += sen
    reviewPos.append(kkma.nouns(all))

In [ ]:
#길이가 2이상인 단어만 저장
reviewPos2=list()
for i in reviewPos:
    all=[]
    for w in i:
        if len(w)>1:
            all.append(w)
    reviewPos2.append(all)

In [ ]:
#길이가 2이상인 단어만 저장
qq2=list()
for i in qq:
    all=[]
    for w in i:
        if len(w)>1:
            all.append(w)
    qq2.append(all)

In [ ]:
from gensim.models import Word2Vec

In [ ]:
# 옵션 설정
num_features = 300  # 문자 벡터 차원 수
min_word_count = 2  # 최소 문자 수
num_workers = 4  # 병렬 처리 스레드 수
context = 5  # 문자열 창 크기
# downsampling = 1e-3  # 문자 빈도수 Downsample



In [ ]:
# kkma로 추출한 word2vec 모델 학습
model = word2vec.Word2Vec(reviewPos2,
                          workers=num_workers,
                          size=num_features,
                          min_count=min_word_count,
                          window=context, sg=0)


In [ ]:
# Komoran로 추출한 word2vec 모델 학습
model2 = word2vec.Word2Vec(qq2,
                          workers=num_workers,
                          size=num_features,
                          min_count=min_word_count,
                          window=context, sg=0)


In [ ]:
print(model)

In [ ]:
print(model2)

In [ ]:
#생성된 words 리스트 확인 
words = list(model.wv.vocab)
print(words)

In [ ]:
#komoran 사용 데이터 
model2.wv.most_similar('의자')

In [ ]:
#kkma 사용 데이터 
model.wv.most_similar('패브릭')

In [ ]:
#kkma 사용 데이터
model.wv.most_similar('스툴')

In [ ]:
#kkma 사용 데이터
model.wv.most_similar('소파')

In [ ]:
#kkma 사용 데이터
model.wv.most_similar('의자')

In [ ]:
!pip install wordcloud

In [ ]:
#wordcloud 생성을 위한 패키지 import 및 한글화 

from wordcloud import WordCloud
import matplotlib.pyplot as plt

from matplotlib import font_manager

font_fname = '../input/textss/NanumGothic.ttf'

font_family = font_manager.FontProperties(fname=font_fname).get_name()

plt.rcParams["font.family"] = font_family

In [ ]:
#중복 의미 단어 stopwords로 지정한 후 wordcloud 정의
stopwords=["만족","총알배송","사용","기사","생각","느낌","구매"]
def displayWordCloud(data = None, backgroundcolor = 'white', width=800, height=768):
    wordcloud = WordCloud(
                        font_path = '../input/textss/NanumGothic.ttf', 
                        stopwords = stopwords, 
                        background_color = backgroundcolor, 
                        width = width, height = height, 
                        random_state=42).generate(data)
                        
    plt.figure(figsize = (15 , 10))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show() 

In [ ]:
#word cloud적용을 위해 텍스트 합침
text=""
for i in reviewPos2:
    text += " ".join(i)
print(text)

In [ ]:
#word cloud적용을 위해 텍스트 합침
text2=""
for i in reviewPos:
    text2 += " ".join(i)

In [ ]:
#word cloud적용을 위해 텍스트 합침
displayWordCloud(text2)

In [ ]:
displayWordCloud(text)

In [ ]:
# 시각화의 한글어 표기를 위한 다양한 시도 
#실패한 시각화
%config InlineBackend.figure_format = 'retina'

from sklearn.decomposition import PCA
from matplotlib import pyplot
import matplotlib.font_manager as fm
fontpath = '../input/textss/NanumGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
'''from sklearn.decomposition import PCA

from matplotlib import pyplot
import matplotlib 
fontpath = '../input/textss/NanumGothic.ttf'
matplotlib.rcParams['axes.unicode_minus'] = False 
plt.rcParams['axes.unicode_minus'] = False 
fontprop = font_manager.FontProperties(fname=fontpath,size=12)

import matplotlib.font_manager as fm
fontpath='../input/textss/NanumGothic.ttf'
font=fm.FontProperties(fname=fontpath, size=9).get_name()
matplotlib.rc('font',family=font) '''

# 단어 벡터들을 PCA를 사용하여 2차원 공간으로 사영(projection)
X = model[model.wv.vocab]
pca = PCA(n_components=2)
result = pca.fit_transform(X)

# 사영한 결과를 그래프로 표시
pyplot.scatter(result[:, 0], result[:, 1])
words = list(model.wv.vocab)
for i, word in enumerate(words):
    pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()